In [1]:
import sys
sys.path.append('/Volumes/Samsung_T5/saturnx')

import numpy as np
from scipy.fftpack import fftfreq, fft

from saturnx.core.lightcurve import Lightcurve
from saturnx.core.gti import Gti
from saturnx.core.power import PowerSpectrum
from saturnx.utils.time_series import poi_events

In [2]:
def fake_white_noise_lc(tres=0.01,nbins=5000,cr=5,low_en=0.5,high_en=10):
    events = poi_events(tres=tres,nbins=nbins,cr=cr)
    time_bin_edges = np.linspace(0,nbins*tres,nbins+1,dtype=np.double)
    time_bins_center = np.linspace(0+tres/2.,nbins*tres-tres/2.,nbins,dtype=np.double)
    hist, dummy = np.histogram(events,time_bin_edges)
    notes = {}
    notes['STEF1'] = 'This is a test note'    
    meta_data = {}
    meta_data['MISSION'] = 'NICER'
    low_en, high_en = 0.5,10
    lc = Lightcurve(time_array = time_bins_center,count_array = hist,
                    low_en=low_en,high_en=high_en,
                    notes=notes, meta_data = meta_data)
    return lc

In [3]:
soft_lc = fake_white_noise_lc(low_en=0.5,high_en=2)
hard_lc = fake_white_noise_lc(low_en=2,high_en=10)
tot_lc =  soft_lc + hard_lc

In [4]:
print(soft_lc.tres,len(soft_lc))

0.01 5000


In [5]:
print(soft_lc.time.iloc[0],soft_lc.time.iloc[-1])

0.005 49.995


In [6]:
tstarts = [0.001,17,30]
tstops  = [16,28,50]
gti = Gti(start_array = tstarts, stop_array = tstops)

In [7]:
gti

,start,stop,dur,gap
0,0.001,16,15.999,0.0
1,17.000,28,11.000,1.0
2,30.000,50,20.000,2.0


In [8]:
soft_gti_lc_list = soft_lc.split(gti)
hard_gti_lc_list = hard_lc.split(gti)
tot_gti_lc_list  = tot_lc.split(gti)

===> Splitting GTI
===> Splitting GTI
===> Splitting GTI


In [12]:
tseg = 4
soft_seg_lc_list = soft_gti_lc_list.split(tseg)
hard_seg_lc_list = hard_gti_lc_list.split(tseg)
tot_seg_lc_list = tot_gti_lc_list.split(tseg)
# soft_seg_lc_list = soft_lc.split(tseg)
# hard_seg_lc_list = hard_lc.split(tseg)
# tot_seg_lc_list = tot_lc.split(tseg)

===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment
===> Splitting Segment


In [13]:
for lc in soft_gti_lc_list:
    print(lc.time.iloc[0],lc.time.iloc[-1],lc.tres)

0.005 15.995 0.01
17.004999999999995 27.994999999999994 0.01
30.004999999999995 49.995 0.01


In [14]:
weights = []
first = True
counter = 0
for lc in soft_seg_lc_list:
    if first:
        first=False
        gti_index = lc.meta_data['GTI_INDEX']
        
    current_gti_index = lc.meta_data['GTI_INDEX']
    if current_gti_index == gti_index:
        counter += 1
    else:
        gti_index = current_gti_index
        weights += [counter]
        counter=1
weights += [counter]
weights = np.array(weights)
print(weights)

[4 2 5]


In [15]:
test_array = np.linspace(0,10,11)

In [16]:
np.pad(test_array,(0,2),mode='constant',constant_values=23)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 23., 23.])

In [17]:
for lc1,lc2 in zip(soft_seg_lc_list,hard_seg_lc_list):
    print(len(lc1),len(lc2))
    fft1 = fft(lc1.counts.to_numpy())
    fft2 = fft(lc2.counts.to_numpy())
    cross = np.multiply(fft1,np.conj(fft2))
    print('cross',len(cross))

400 400
cross 400
400 400
cross 400
400 400
cross 400
400 400
cross 400
400 400
cross 400
400 400
cross 400
400 400
cross 400
400 400
cross 400
400 400
cross 400
400 400
cross 400
400 400
cross 400


In [27]:
def comp_cross(lc1,lc2):
    fft1 = fft(lc1.counts.to_numpy())
    fft2 = fft(lc2.counts.to_numpy())
    cross = np.multiply(fft1,np.conj(fft2))
    return cross

def comp_cross_err(pw1,pw2,cross,n):
    re2 = (np.real(cross))**2
    im2 = (np.imag(cross))**2
    dre = np.sqrt( (pw1.power*pw2.power + re2 - im2) / 2 / n )
    dim = np.sqrt( (pw1.power*pw2.power - re2 + im2) / 2 / n )
    return dre,dim

def comp_cross_ave(lc_list1,lc_list2):
    first = True
    for lc1, lc2 in zip(lc_list1,lc_list2):
        if first:
            first = False
            cross = comp_cross(lc1,lc2)
        else:
            cross += comp_cross(lc1,lc2)
    average_cross = cross/len(lc_list1)
    
    pw_list1 = PowerSpectrum.from_lc(lc_list1)
    pw_list2 = PowerSpectrum.from_lc(lc_list2)
    
    pw1 = pw_list1.average(norm=None)
    pw2 = pw_list2.average(norm=None)
    
    dre,dim = comp_cross_err(pw1,pw2,cross,len(lc_list1))
    return cross, dre, dim

def comp_weighted_cross_ave(cross_array,dre_array,dim_array,weights):
    assert len(cross_array) == len(dre_array)
    assert len(cross_array) == len(dim_array)
    assert len(cross_array) == len(weights)
    
    ave_cross = cross_array[0]*weights[0]
    ave_dre2 = dre_array[0]**2*weights[0]
    ave_dim2 = dim_array[0]**2*weights[0]
    for i in range(1,len(cross_array)):
        ave_cross += cross_array[i]*weights[i]
        ave_dre2 += dre_array[i]**2*weights[i]
        ave_dim2 += dim_array[i]**2*weights[i]
    ave_cross = ave_cross / np.sum(weigts)
    ave_dre2 = ave_dre2 / np.sum(weights)
    ave_dim2 = ave_dim2 / np.sum(weights)
    
    return ave_cross, np.sqrt(ave_dre2), np.sqrt(ave_dim2)

In [28]:
ave_cross_from_seg, dre_from_seg, dim_from_seg = comp_cross_ave(soft_seg_lc_list, hard_seg_lc_list)

In [29]:
cross_gti = []
dre_gti = []
dim_gti = []
for lc1,lc2 in zip(soft_gti_lc_list,hard_gti_lc_list):
    tmp_cross, tmp_dre, tmp_dim = comp_cross_ave(lc1,lc2)
    cross_gti += [tmp_cross]   
    dre_gti += [tmp_dre]
    dim_gti += [tmp_dmi]
ave_cross_from_gti, dre_from_gti, dim_from_gti = comp_weighted_cross_ave(cross_gti, dre_gti, dim_gti)
    

AttributeError: 'str' object has no attribute 'counts'

In [20]:
for lc in tot_seg_lc_list:
    print(lc.texp)

5.000000000000004
4.999999999999893
4.999999999999893
4.99000000000078
4.99000000000078
4.999999999999005
4.999999999999005
4.999999999999005
4.999999999999005
